In [ ]:
from collections.abc import Sequence
from typing import Iterable, Any
from itertools import count, islice
import unittest


class Reversed[Any](Sequence):
    def __init__(self, iterable: Iterable[Any], max_size: int = 1000):
        self._lst = list(reversed(list(islice(iterable, max_size))))
        self._iter = iter(self._lst)

    def __iter__(self) -> "Reversed":
        return self

    def __next__(self) -> Any:
        return next(self._iter)

    def __getitem__(self, index: int) -> Any:
        return self._lst[index]

    def __len__(self) -> int:
        return len(self._lst)


class TestReversed(unittest.TestCase):
    def test_sequence(self):
        r = Reversed(iterable="abc")
        assert list(islice(r, 3)) == ["c", "b", "a"]

    def test_raises_stop_iteration(self):
        r = Reversed(iterable="abc")
        assert list(islice(r, 3)) == ["c", "b", "a"]
        with self.assertRaises(StopIteration):
            next(r)

    def test_empty(self):
        r = Reversed(iterable=())
        assert list(islice(r, 3)) == []

    def test_iterator(self):
        r = Reversed(iterable=iter([1, 2, 3]))
        assert list(islice(r, 2)) == [3, 2]

    def test_generator(self):
        r = Reversed(iterable=(_ for _ in range(10001)))
        assert list(islice(r, 4)) == [999, 998, 997, 996]

    def test_limited(self):
        r = Reversed(iterable=iter([1, 2, 3]), max_size=1)
        assert list(islice(r, 2)) == [1]

    def test_inf_generator(self):
        r = Reversed(iterable=count(0, 1))
        assert list(islice(r, 4)) == [999, 998, 997, 996]

    def test_len(self):
        r = Reversed(iterable=count(0, 1))
        assert len(r) == 1000
        assert list(islice(r, 4)) == [999, 998, 997, 996]

    def test_getitem(self):
        r = Reversed(iterable=count(0, 1))
        assert r[-1] == 0
        assert list(islice(r, 4)) == [999, 998, 997, 996]


if __name__ == "__main__":
    unittest.TextTestRunner(verbosity=2).run(
        unittest.TestLoader().loadTestsFromTestCase(testCaseClass=TestReversed)
    )

In [ ]:
from typing import Iterable, Any
from itertools import islice


def inf_gen(iterable: Iterable[Any]) -> Iterable[Any]:
    lst = []
    try:
        for elem in iterable:
            lst.append(elem)
            yield elem
    except TypeError:
        raise ValueError("Non-iterable input object")

    while lst:
        yield from lst


class TestInfiniteGenerator(unittest.TestCase):
    def test_sequence(self):
        generator = inf_gen(iterable="abc")
        assert list(islice(generator, 6)) == ["a", "b", "c", "a", "b", "c"]

    def test_empty(self):
        generator = inf_gen(iterable=())
        assert list(generator) == []

    def test_non_iterable(self):
        with self.assertRaises(ValueError):
            generator = inf_gen(iterable=5)
            next(generator)

    def test_iterator(self):
        generator = inf_gen(iterable=iter("abc"))
        assert list(islice(generator, 6)) == ["a", "b", "c", "a", "b", "c"]

    def test_generator(self):
        generator = inf_gen(iterable=(i for i in range(3)))
        assert list(islice(generator, 6)) == [0, 1, 2, 0, 1, 2]

    def test_inf_generator(self):
        generator = inf_gen(iterable=inf_gen([1]))
        assert list(islice(generator, 6)) == [1, 1, 1, 1, 1, 1]


if __name__ == "__main__":
    unittest.TextTestRunner(verbosity=2).run(
        unittest.TestLoader().loadTestsFromTestCase(testCaseClass=TestInfiniteGenerator)
    )

In [3]:
from typing import Generator
import unittest
from enum import Enum


class Operation(Enum):
    ADD = "add"
    MULTIPLY = "multiply"
    SUBTRACT = "subtract"
    DIVIDE = "divide"
    UNKNOWN = "unknown"


def calculator(
    start_value: float = 0.0,
) -> Generator[float, tuple[Operation, float], None]:
    value = start_value
    while True:
        try:
            operation, number = yield value
            match operation:
                case Operation.ADD:
                    value += number
                case Operation.MULTIPLY:
                    value *= number
                case Operation.SUBTRACT:
                    value -= number
                case Operation.DIVIDE:
                    if number == 0:
                        raise ZeroDivisionError("Cannot divide by zero")
                    value /= number
                case _:
                    raise ValueError(f"Unknown operation: {operation.value}")
        except GeneratorExit:
            return
        except (ZeroDivisionError, ValueError):
            raise
        except Exception:
            ...


class TestCalculator(unittest.TestCase):
    def setUp(self):
        """Инициализация калькулятора перед каждым тестом"""
        self.calc = calculator(10)
        next(self.calc)

    def test_calculator_basic_operations(self):
        """Тест базовых операций калькулятора"""
        self.assertEqual(self.calc.send((Operation.ADD, 5)), 15)
        self.assertEqual(self.calc.send((Operation.MULTIPLY, 3)), 45)
        self.assertEqual(self.calc.send((Operation.SUBTRACT, 15)), 30)
        self.assertEqual(self.calc.send((Operation.DIVIDE, 2)), 15)

    def test_calculator_zero_division(self):
        """Тест деления на ноль"""
        with self.assertRaises(ZeroDivisionError) as context:
            self.calc.send((Operation.DIVIDE, 0))
        self.assertEqual(str(context.exception), "Cannot divide by zero")

    def test_calculator_unknown_operation(self):
        """Тест неизвестной операции"""
        with self.assertRaises(ValueError) as context:
            self.calc.send((Operation.UNKNOWN, 5))
        self.assertEqual(str(context.exception), "Unknown operation: unknown")

    def test_calculator_close(self):
        """Тест закрытия калькулятора"""
        self.calc.close()
        with self.assertRaises(StopIteration):
            next(self.calc)

    def test_calculator_throw(self):
        """Тест принудительного исключения"""
        self.calc.throw(AttributeError("Manual throw"))


if __name__ == "__main__":
    unittest.TextTestRunner(verbosity=2).run(
        unittest.TestLoader().loadTestsFromTestCase(testCaseClass=TestCalculator)
    )

test_calculator_basic_operations (__main__.TestCalculator.test_calculator_basic_operations)
Тест базовых операций калькулятора ... ok
test_calculator_close (__main__.TestCalculator.test_calculator_close)
Тест закрытия калькулятора ... ok
test_calculator_throw (__main__.TestCalculator.test_calculator_throw)
Тест принудительного исключения ... ok
test_calculator_unknown_operation (__main__.TestCalculator.test_calculator_unknown_operation)
Тест неизвестной операции ... ok
test_calculator_zero_division (__main__.TestCalculator.test_calculator_zero_division)
Тест деления на ноль ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.004s

OK
